In [1]:
import requests 
import numpy as np
import pandas as pd
from datetime import datetime
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
api = 'http://api.cal-adapt.org/api'
headers = {'ContentType': 'json'}

# Your point of interest
point = 'POINT(-122.2657 38.0389)'

# Name of boundary layer in API
resource = 'hydrounits'

# Request url
url = api + '/' + resource + '/'
# Request params to find intersecting polygon
params = {'intersects': point, 'srs': 4326, 'simplify': .0001, 'precision': 4}
ref = ''
location = ''

# Get geometry
response = requests.get(url, params=params, headers=headers)
if response.ok:
    data = response.json()
    print(data)
    feature = data['features'][0]
    if (feature):
        ref = '/api/%s/%s/' % (resource, feature['id'])
        #location = feature['properties']['name']
        #print('ref =', ref)
        #print('location =', location)
    else:
        print('Did not find any polygons that intersect your point')
params = {'ref': ref, 'stat': 'mean'}
#print(data)

{'count': 1, 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}}, 'features': [{'geometry': {'type': 'Polygon', 'coordinates': [[[-122.2815541633435, 38.27869348445772], [-122.2828988831327, 38.27457366779746], [-122.2832453831322, 38.27402221571502], [-122.2836632789649, 38.27375617509043], [-122.2835310425066, 38.273615677174], [-122.2836102383396, 38.27333678134104], [-122.2844078112557, 38.27273783550869], [-122.28454290188, 38.27236915946758], [-122.2853986664619, 38.27202741050973], [-122.2862865518771, 38.27110456155288], [-122.2860034872944, 38.27184634801001], [-122.2851755789628, 38.27291826155006], [-122.2860752070862, 38.27225684071773], [-122.286706002919, 38.27124680634428], [-122.2866985112516, 38.27062491676198], [-122.2861202122948, 38.26955795113863], [-122.2858130841696, 38.26794288759948], [-122.2856591050032, 38.2685637938485], [-122.2858093550033, 38.26890717613962], [-122.2858008727115, 38.2697002000967], [-122.2848418268801, 38.26979348

In [3]:
def get_projections():
    # Create an empty list to hold dataframes
    df_list = []

    # Make a combined list of GCMs, scenarios, climate variables for looping
    climvar = ['precip']
    period = ['day']
    models = ['CanESM2', 'CNRM-CM5', 'HadGEM2-ES', 'MIROC5']
    scenarios = ['historical','rcp85']
    zipped = itertools.product(climvar, period, models, scenarios)
    
    # Request parameters
    params = {'ref': ref, 'stat': 'mean'}

    # Loop through zipped
    for item in zipped:
        # Create slug
        slug = '_'.join(item)
        df = process_daily_data(slug, params)
        df_list.append(df)
            
    # Combine all the dataframes into one and return
    return pd.concat(df_list)

In [4]:
# Cal-Adapt API
api = 'http://api.cal-adapt.org/api'

# Request header
headers = {'ContentType': 'json'}

def process_daily_data(slug, params):
    # Make request
    url = api + '/series/' + slug + '/rasters/'
    response = requests.get(url, params=params)
    
    # Get data
    if response.ok:
        #print('Processing:', slug)
        json = response.json()
        data = json['results'][0]
        # Multiband raster data is returned by the API as a 3D array having a shape like (233376, 1, 1)
        # Flatten the 3D array into a 1D array
        values_arr = np.array(data['image'])
        values_arr = values_arr.flatten()# Get total number of values -> number of days
        length = len(values_arr)
        # Get start date of timeseries
        start_date = datetime.strptime(data['event'], '%Y-%m-%d')
        
        # Create new pandas dataframe and map each value in list to a date index
        df = pd.DataFrame(
            values_arr,
            index= pd.date_range(start_date, freq='1D', periods=length),
            columns=['value']
        )    
        #units = data['units']
        #print("Units are " + units)
        # Add slug as a column
        df['slug'] = slug  

    else:
        print('Failed to download:', slug)
            
    # Combine all the dataframes into one and return
    return df

In [5]:
def get_projections():
    # Create an empty list to hold dataframes
    df_list = []

    # Make a combined list of GCMs, scenarios, climate variables for looping
    climvar = ['pr']
    period = ['day']
    models = ['CanESM2', 'CNRM-CM5', 'HadGEM2-ES', 'MIROC5']
    scenarios = ['historical','rcp85']
    zipped = itertools.product(climvar, period, models, scenarios)
    
    # Request parameters
    params = {'ref': ref, 'stat': 'mean'}

    # Loop through zipped
    for item in zipped:
        # Create slug
        slug = '_'.join(item)
        df = process_daily_data(slug, params)
        df_list.append(df)
            
    # Combine all the dataframes into one and return
    return pd.concat(df_list)

In [6]:
projectedDF = get_projections()
projectedDF.head(10)


,value,slug
1950-01-01,0.000459,pr_day_CanESM2_historical
1950-01-02,0.000266,pr_day_CanESM2_historical
1950-01-03,0.000051,pr_day_CanESM2_historical
1950-01-04,0.000106,pr_day_CanESM2_historical
1950-01-05,0.000064,pr_day_CanESM2_historical
1950-01-06,0.000000,pr_day_CanESM2_historical
1950-01-07,0.000000,pr_day_CanESM2_historical
1950-01-08,0.000000,pr_day_CanESM2_historical
1950-01-09,0.000008,pr_day_CanESM2_historical
1950-01-10,0.000008,pr_day_CanESM2_historical


In [7]:
projectedDF.tail(10)

,value,slug
2099-12-22,0.000000e+00,pr_day_MIROC5_rcp85
2099-12-23,2.647433e-06,pr_day_MIROC5_rcp85
2099-12-24,3.970425e-10,pr_day_MIROC5_rcp85
2099-12-25,2.486744e-06,pr_day_MIROC5_rcp85
2099-12-26,5.075890e-05,pr_day_MIROC5_rcp85
2099-12-27,2.161379e-05,pr_day_MIROC5_rcp85
2099-12-28,1.798033e-06,pr_day_MIROC5_rcp85
2099-12-29,5.365818e-09,pr_day_MIROC5_rcp85
2099-12-30,0.000000e+00,pr_day_MIROC5_rcp85
2099-12-31,0.000000e+00,pr_day_MIROC5_rcp85


In [8]:
temp = projectedDF['slug'].str.split('_', expand=True)[2]
projectedDF['test'] = temp

In [9]:
projectedDF.drop(columns= 'slug', inplace = True)
projectedDF.head()

,value,test
1950-01-01,0.000459,CanESM2
1950-01-02,0.000266,CanESM2
1950-01-03,0.000051,CanESM2
1950-01-04,0.000106,CanESM2
1950-01-05,0.000064,CanESM2


In [10]:
projectedDF.index.name = 'Date'

In [11]:
projectedDF = projectedDF.pivot_table( values = 'value', index = 'Date', columns = 'test')
projectedDF.head(50)

test,CNRM-CM5,CanESM2,HadGEM2-ES,MIROC5
Date,,,,
1950-01-01,3.919359e-06,4.592555e-04,9.844477e-08,0.000000e+00
1950-01-02,0.000000e+00,2.661875e-04,1.116732e-05,0.000000e+00
1950-01-03,0.000000e+00,5.118674e-05,4.794532e-06,1.016732e-05
1950-01-04,0.000000e+00,1.062064e-04,0.000000e+00,7.368240e-10
1950-01-05,0.000000e+00,6.440178e-05,2.225221e-07,0.000000e+00
1950-01-06,0.000000e+00,0.000000e+00,1.418880e-07,0.000000e+00
1950-01-07,7.242959e-06,0.000000e+00,1.722287e-05,0.000000e+00
1950-01-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1950-01-09,3.176270e-05,7.714196e-06,4.809488e-05,0.000000e+00


In [12]:
#projectedDF[] = projectedDF.iloc[0:4].apply(lambda x: x * 86400)
projectedDF = projectedDF.apply(lambda x: x*86400)

In [14]:
projectedDF.head(400)

test,CNRM-CM5,CanESM2,HadGEM2-ES,MIROC5
Date,,,,
1950-01-01,0.338633,39.679677,0.008506,0.000000
1950-01-02,0.000000,22.998604,0.964856,0.000000
1950-01-03,0.000000,4.422534,0.414248,0.878456
1950-01-04,0.000000,9.176232,0.000000,0.000064
1950-01-05,0.000000,5.564314,0.019226,0.000000
1950-01-06,0.000000,0.000000,0.012259,0.000000
1950-01-07,0.625792,0.000000,1.488056,0.000000
1950-01-08,0.000000,0.000000,0.000000,0.000000
1950-01-09,2.744297,0.666507,4.155398,0.000000
